In [1]:
import csv
import json
import base64
from spire.pdf import *

from urllib.request import urlopen

key = "55228d9d4f216ad5972abd1be3c4c2e3"

In [2]:
def get_bill_ids(filePath):
    """Get bill ids from a csv file"""

    bill_ids = []
    with(open(filePath, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            bill_ids.append(row[0])
    return bill_ids        

In [4]:
VA_bill_ids = get_bill_ids("VA/2024-2024_Regular_Session/csv/bills.csv")
PA_bill_ids = get_bill_ids("PA/2023-2024_Regular_Session/csv/bills.csv")
MN_bill_ids = get_bill_ids("MN/2023-2024_93rd_Legislature/csv/bills.csv")

In [5]:
VA_bill_ids[:10]

['bill_id',
 '1877734',
 '1877735',
 '1882815',
 '1883989',
 '1877732',
 '1877737',
 '1877730',
 '1877736',
 '1877727']

In [5]:
def get_text_from_bill_id(bill_id):
    """API pull to get text from bill id
    returns None if bill id is not found
    returns empy string if no text is unavailable
    returns encoded text (with base64) if available
    """
    
    urlGetBill = f"https://api.legiscan.com/?key={key}&op=getBill&id={bill_id}"
    r1 = urlopen(urlGetBill).read().decode()
    data1 = json.loads(r1)
    if data1["status"] == "ERROR":
        return None
    if data1["bill"]["texts"] == []:
        return ""
    else:
        doc_id = data1["bill"]["texts"][0]["doc_id"]
        urlGetText = f"https://api.legiscan.com/?key={key}&op=getBillText&id={doc_id}"
        r2 = urlopen(urlGetText).read().decode()
        data2 = json.loads(r2)
        textEncoded = data2["text"]["doc"]
        return textEncoded

In [7]:
PA_bill_ids[4]

'1745716'

In [13]:
data = get_text_from_bill_id(PA_bill_ids[240])
base64.b64decode(data)
stream = Stream(base64.b64decode(data))
pdf = PdfDocument(stream)
textExtractor = PdfTextExtractor(pdf)
# extractOptions = PdfTextExtractOptions()
# extractOptions.IsExtractAllText = True
# text = textExtractor.ExtractText(extractOptions)
# print(text)
# with open("sampleText.txt", "w") as f:
#     f.write(data)

# with open("sampleText.txt", "w") as f:

#     f.write(base64.b64decode(data).decode())

SpireException: Arg_InvalidCastException:   at System.Runtime.TypeCast.CheckCastClass(MethodTable*, Object) + 0x31
   at Spire.AOT.Helper`1.PtrToObject(IntPtr) + 0xd6
   at Spire.Pdf.AOT.NLPdfTextExtractor.PdfTextExtractor_CreatePdfTextExtractorP(IntPtr, IntPtr) + 0x53

In [8]:
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["state","bill_id", "text"])
count = 0
for bill_id in VA_bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    count += 1
    if count % 100 == 0:
        print(count)
    bill_texts.append(["VA", bill_id, billText]) #If empty string then text was unavailable on LegiScan

# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    
with open("bill_texts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)
        


100
200
